## 1. Import packages

In [1]:
pip install tweepy

Note: you may need to restart the kernel to use updated packages.


In [178]:
import os
import tweepy as tw
import pandas as pd
from tqdm import tqdm, notebook

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
pd.set_option('display.max_colwidth', None)
pd.set_option('display.width', None)

## 2. Twitter API authentication

In [179]:
consumer_api_key = 'AphvFIN921yxdHCdVaTxVXwIT'
consumer_api_secret = 'l22IYQxDZmOzPLTx2PBHt6JQBlpcZiNpHB3JaO0EwsBxZvv342'

In [180]:
auth = tw.OAuthHandler(consumer_api_key, consumer_api_secret)

In [181]:
api = tw.API(auth, wait_on_rate_limit=True)

## 3. Tweets query

### 3.1. Define the query

In [182]:
tweets = tw.Cursor(api.search_tweets,q='#covid19 -filter:retweets',lang='en').items(30000)


### 3.2. Retreive the tweets

In [183]:
#sleep_on_rate_limit=False
tweets_copy = []
for tweet in tqdm(tweets):
     tweets_copy.append(tweet)

6464it [01:46, 57.20it/s]Rate limit reached. Sleeping for: 796
13040it [16:47, 56.07it/s]Rate limit reached. Sleeping for: 798
19675it [32:02, 69.20it/s]Rate limit reached. Sleeping for: 784
24352it [46:32,  8.72it/s]


In [184]:
print(f"new tweets retrieved: {len(tweets_copy)}")

new tweets retrieved: 24352


## 4. Populate the dataset

In [185]:
tweets_df = pd.DataFrame()
for tweet in tqdm(tweets_copy):
    hashtags = []
    try:
        for hashtag in tweet.entities["hashtags"]:
            hashtags.append(hashtag["text"])
        text = api.get_status(id=tweet.id, tweet_mode='extended').full_text
    except:
        pass
    tweets_df = tweets_df.append(pd.DataFrame({'user_name': tweet.user.name, 
                                               'user_location': tweet.user.location,\
                                               'user_description': tweet.user.description,
                                               'user_created': tweet.user.created_at,
                                               'user_followers': tweet.user.followers_count,
                                               'user_friends': tweet.user.friends_count,
                                               'user_favourites': tweet.user.favourites_count,
                                               'user_verified': tweet.user.verified,
                                               'date': tweet.created_at,
                                               'text': text, 
                                               'hashtags': [hashtags if hashtags else None],
                                               'source': tweet.source,
                                               'is_retweet': tweet.retweeted}, index=[0]))

100%|██████████| 24352/24352 [6:45:41<00:00,  1.00it/s]   


In [186]:
tweets_df.tail()

,user_name,user_location,user_description,user_created,user_followers,user_friends,user_favourites,user_verified,date,text,hashtags,source,is_retweet
0,dixonsgtusmc@yahoo.com,WYOMING,"Human mind kind OF miniature representation_aeons_emanate from_ultimate God… reason, Gnostic could also contemplate God by contemplating HER-his own intellect!!",2021-10-15 04:25:57+00:00,889,1452,2021,False,2023-01-29 23:32:24+00:00,"1's Tweets SAYING THAT LONGER,that why DOJ FBI FDA REFUSE 2 RELEASE-BEST ANSWER @techreview be APART KEEPING BEST GOD's CREATED HERB BEING APPROVE! \nDr. Thomas Ichim\n@exosome $RVVTF makes #bucillamine 4 #COVID19 \n#coronavirus messes up endothelial cells https://t.co/qrvJoQ7rmT..",None,Twitter Web App,False
0,COVID Scale,Earth,A bot designed to put the #COVID19 pandemic's effect on the United States into historical perspective. Stats from @Worldometers,2017-09-15 19:54:39+00:00,1342,3,6,False,2023-01-29 23:30:41+00:00,"Since the start of the pandemic, 1,132,256 Americans have died from #COVID19 (16.8% of all deaths worldwide). That is about the same as 3,560 1993 Storm of the Centurys which killed 318 people in 1993: https://t.co/rwPHiBdxoj",[COVID19],HN_Comments,False
0,TruthWins1984,The Free World Order,"Long time critical thinker. Stay awake not woke, and just say no to #CBDC.",2023-01-29 21:38:04+00:00,5,44,27,False,2023-01-29 23:20:14+00:00,This type of BS propaganda would be absolutely laughable if it wasn't so deliberately evil.\n\n#WEF #pandemic #COVID19 #MSM https://t.co/wRfEXSns5v,"[WEF, pandemic]",Twitter Web App,False
0,The Epoch Times,"New York, USA","Independent, award-winning reporting based on Truth and Tradition. Newsletters: https://t.co/0F3JANwb8P | Watch: https://t.co/M1G2gEVJmq | Play: https://t.co/pDrVrN4Qcy",2009-04-05 23:54:55+00:00,576863,101,56174,True,2023-01-29 22:50:30+00:00,The #CCP never ceased its persecution of #FalunGong adherents during its 3 years of #COVID19 lockdowns.\n\n2 elderly Falun Gong adherents recently lost their lives after years of suffering under persecution from the regime. https://t.co/345l3QTjmS,"[CCP, FalunGong, COVID19]",Twitter Web App,False
0,DerekKissach,Sussex,Politics is about lying to us and sneering at us from an invented moral highground. Brexit and Trump is the result.,2009-08-17 01:06:33+00:00,94,114,21376,False,2023-01-29 22:48:19+00:00,"@MikeHudema #COVID19 #Pfizer #HarrisonFord _ The actor Harrison Ford, proving actors need a script. Or has he just crashed another plane?\nWho knows.","[COVID19, Pfizer, HarrisonFord]",Twitter Web App,False


In [187]:
tweets_df.to_csv("covid19_2023.csv", index=False)

## 5. Save the data

### 5.1. Read past data

In [11]:
tweets_old_df = pd.read_csv("covid19_tweets.csv")
print(f"past tweets: {tweets_old_df.shape}")

past tweets: (168264, 13)


/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:3020: DtypeWarning: Columns (7,12) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


### 5.2. Merge past and present data

In [12]:
tweets_all_df = pd.concat([tweets_old_df, tweets_df], axis=0)
print(f"new tweets: {tweets_df.shape[0]} past tweets: {tweets_old_df.shape[0]} all tweets: {tweets_all_df.shape[0]}")

new tweets: 12452 past tweets: 168264 all tweets: 180716


### 5.3. Drop duplicates

In [13]:
tweets_all_df.drop_duplicates(subset = ["user_name", "date", "text"], inplace=True)
print(f"all tweets: {tweets_all_df.shape}")

all tweets: (180716, 13)


### 5.4. Export the updated data

In [14]:
tweets_all_df.to_csv("covid19_tweets.csv", index=False)